# Data Preprocessing
코드 공유에 올라와있는 EDA_공유.ipynb를 참고하여 전처리를 진행하였습니다.

In [ ]:
# colab 한글 폰트 사용 관련 설정
# reference to https://velog.io/@winocas/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%9C%EA%B0%81%ED%99%94-%ED%95%9C%EA%B8%80%EB%A7%88%EC%9D%B4%EB%84%88%EC%8A%A4-%EA%B9%A8%EC%A7%90-%EC%9C%88%EB%8F%84%EC%9A%B0

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 해당 셀 실행 후 런타임 다시 시작(단축키: Ctrl+M+.)을 해야합니다.

In [ ]:
!pip install dataprep
!pip install konlpy
!pip install transformers
!pip install sentencepiece

# tokenizer 활용 위한 clone
!git clone https://github.com/SKTBrain/KoBERT.git

In [3]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
import os

In [4]:
os.chdir("./KoBERT/kobert_hf")
PATH = '../../../data/'
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'test.csv')

In [5]:
train_data = train.loc[~train.duplicated(keep='first')].copy()
test_data = test.loc[~test.duplicated(keep='first')].copy()

drop_list = ['보안과제정보', '평가관리비', '0', '·', '보안사항으로 생략', '비밀과제로 보안상 생략', '비밀사업으로 보안상 생략']

train_data = train_data.drop(train_data.query("요약문_연구내용 in @drop_list").index)
test_data = test_data.drop(test_data.query("요약문_연구내용 in @drop_list").index)

In [6]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})

# tokenizer가 [unk]로 처리하는 특수문자 확인
special_char = []
col_names = ['요약문_연구목표', '요약문_연구내용', '요약문_기대효과']
for col_name in col_names:
    for text in train[col_name]:
        spec_list = re.findall("[^가-힣a-zA-Z0-9\n\(^u) ]", str(text))
        for spe in spec_list:
            if spe in special_char:
                continue
            else:
                special_char.append(spe)
            
unk_spec_char = []
for spe in special_char:
    if tokenizer.unk_token_id in tokenizer.encode(spe):
        if spe in unk_spec_char:
            continue
        else:
            unk_spec_char.append(spe)

In [7]:
def text_preprocessing(text):
    text = str(text)
    # 특수문자를 모두 \n으로 바꿔준다.
    text = re.sub("(\n)+(\s)*\W","\n", text)

    # 맨 앞에 남아있는 white space를 지워준다.
    if re.match("\w", text) is None:
        text = text[1:]

    # \n으로 구분되어있는 문자들을 잘라서 리스트에 넣어준다.
    sentence_list = text.split("\n")
    
    # 잘린 문자열을 처리해서 상용 글자만 남겨둔다.
    for sent_idx in range(len(sentence_list)):
        re_sent = ''.join([char for char in sentence_list[sent_idx] if char not in set(unk_spec_char)])
        # 맨 앞에 띄어쓰기가 남아있다면 지워준다.
        if len(re_sent) > 0 and re_sent[0] == ' ':
            re_sent = re_sent[1:]
        # 리스트에 처리가 완료된 문자열을 넣어준다.
        sentence_list[sent_idx] = re_sent
    
    # 하나의 리스트에 들어있는 문장들을 [SEP] 토큰으로 분류하여 하나의 문자열로 만들어준다.
    pre_str = sentence_list.pop(0)
    for s in sentence_list:
        pre_str = pre_str + " [SEP] " + s
    # 전처리 완료된 문자열 반환
    return pre_str

In [8]:
keys = ['요약문_연구목표', '요약문_연구내용', '요약문_기대효과'] # 전처리 할 column
# train data 전처리
for key in (keys):
    temp = []
    for sent in tqdm(list(train_data[key])):
        temp.append(text_preprocessing(sent))
    train_data[key] = temp
# test data 전처리
for key in (keys):
    temp = []
    for sent in tqdm(list(test_data[key])):
        temp.append(text_preprocessing(sent))
    test_data[key] = temp
print('FINISH')

100%|██████████| 43364/43364 [11:53<00:00, 60.74it/s]

FINISH


In [9]:
train_data.to_csv(PATH + 'train_preprocessing.csv', index=False)
test_data.to_csv(PATH + 'test_preprocessing.csv', index=False)